In [23]:
import os
os.chdir("End-End DS Project")
%pwd

'd:\\End-End DS Project'

In [24]:
import dagshub
dagshub.init(repo_owner='abheshith7', repo_name='WineQuality-Prediction-Pipeline', mlflow=True)

[2025-04-17 12:15:18,305: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/abheshith7/WineQuality-Prediction-Pipeline "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "abheshith7/WineQuality-Prediction-Pipeline"

[2025-04-17 12:15:18,312: INFO: helpers: Initialized MLflow to track repo "abheshith7/WineQuality-Prediction-Pipeline"]


Repository abheshith7/WineQuality-Prediction-Pipeline initialized!

[2025-04-17 12:15:18,327: INFO: helpers: Repository abheshith7/WineQuality-Prediction-Pipeline initialized!]


In [25]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/abheshith7/WineQuality-Prediction-Pipeline.mlflow"

In [26]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    model_path: Path
    test_data_path: Path
    all_params: dict
    metrics_file_name: Path
    target_column: str
    mlflow_uri: str

In [27]:
from src.DataScience.constants import *
from src.DataScience.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, 
                       params_filepath = PARAMS_FILE_PATH, 
                       schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)


        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        schema = self.schema.TARGET_COLUMN
        params = self.params.Elasticnet

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            model_path = config.model_path,
            test_data_path = config.test_data_path,
            all_params = params,
            metrics_file_name = config.metrics_file_name,
            target_column = schema.name,
            mlflow_uri =  "https://dagshub.com/abheshith7/WineQuality-Prediction-Pipeline.mlflow"
        )
        return model_evaluation_config

In [28]:
from src.DataScience import logger
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
from mlflow.models import infer_signature
import joblib
import os
from urllib.parse import urlparse

In [31]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        X_test = test_data.drop(self.config.target_column, axis=1)
        y_test = test_data[self.config.target_column]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predictions = model.predict(X_test)

            (rmse, mae, r2) = self.eval_metrics(y_test, predictions)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            # Save metrics as a local JSON file
            metrics = {
                "rmse": rmse,
                "mae": mae,
                "r2": r2
            }
            save_json(Path(self.config.metrics_file_name), metrics)

            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNetModel")
            else:
                mlflow.sklearn.log_model(model, "model")
                
            

In [32]:
try:
    config = ConfigurationManager()
    model_eval_config = config.get_model_evaluation_config()
    model_eval = ModelEvaluation(model_eval_config)
    model_eval.log_into_mlflow()
except Exception as e:
    logger.exception(e)
    raise e

[2025-04-17 12:16:34,968: INFO: common: yaml file: config\config.yaml loaded sucessfully]
[2025-04-17 12:16:34,971: INFO: common: yaml file: params.yaml loaded sucessfully]
[2025-04-17 12:16:34,974: INFO: common: yaml file: schema.yaml loaded sucessfully]
[2025-04-17 12:16:34,975: INFO: common: Creating a Directory at: artifacts]
[2025-04-17 12:16:34,978: INFO: common: Creating a Directory at: artifacts/model_evaluation]
[2025-04-17 12:16:36,994: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


2025/04/17 12:16:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'ElasticNetModel' already exists. Creating a new version of this model...
2025/04/17 12:16:46 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticNetModel, version 2
Created version '2' of model 'ElasticNetModel'.


🏃 View run bald-stork-680 at: https://dagshub.com/abheshith7/WineQuality-Prediction-Pipeline.mlflow/#/experiments/0/runs/1972c6fbcc7949f58451803d79ed6183
🧪 View experiment at: https://dagshub.com/abheshith7/WineQuality-Prediction-Pipeline.mlflow/#/experiments/0
